<a href="https://colab.research.google.com/github/maderix/covid_analysis/blob/master/reddit_covid_positive_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw
!pip install tf-nightly

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy

In [0]:
 import praw
 reddit = praw.Reddit(client_id='JlcL0XKfkM6fBw', client_secret='th1yQxpTkAftyan22fiN4sH9nuA', user_agent='covidscraper')

In [0]:
covid_desc = []
covid_scores = []
for submission in reddit.subreddit('COVID19positive').search('flair:"Tested Positive - Me"', limit=1000):
    covid_desc.append(submission.selftext)
    covid_scores.append(submission.score)
covid_scores = numpy.array(covid_scores)
covid_scores = covid_scores / numpy.max(covid_scores)

In [0]:
import numpy
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((covid_desc,covid_scores))

In [6]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set()
for text_tensor, _ in dataset:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

7733

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [0]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.float64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


encoded_data = dataset.map(encode_map_fn)

In [10]:
TAKE_SIZE = 16
BUFFER_SIZE = 256
BATCH_SIZE = 4
train_data = encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))

test_data = encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None],[]))
len(list(test_data))

4

In [11]:
sample_text, sample_labels = next(iter(test_data))
sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(321,), dtype=int64, numpy=
 array([1494, 3609, 7396, 2197, 5119, 2695, 6708,  490, 6352, 5709, 3081,
        3028, 5055, 2511,  490, 6858, 4863, 3672, 7046, 6409, 3291, 6293,
        2125, 5770,  589, 5148, 3726, 5204, 3259, 4442,  490, 2575, 1765,
        6897, 7433,  929,  929, 6554, 3949, 6703, 2054, 1489, 1128, 6257,
        2047, 3289, 6858, 4295, 4876, 1348, 7543, 6858, 4631, 5741, 7416,
        1599, 1112, 2329, 1489, 5562, 2064, 6429, 6858, 4865,   62,  642,
        1489, 6409, 5773, 7479, 7461, 4325, 3028, 5357, 1489,  495, 4876,
        5034,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    

In [0]:
vocab_size += 1

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
# One or more dense layers.
# Edit the list in the `for` line to experiment with layer sizes.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))
model.compile(optimizer='adam',
              loss='mse')

In [14]:
model.fit(train_data, epochs=10, validation_data=test_data)


Epoch 1/10
57/57 [==============================] - 4s 71ms/step - loss: 0.0120 - val_loss: 0.2614
Epoch 2/10
57/57 [==============================] - 3s 59ms/step - loss: 0.0100 - val_loss: 0.2553
Epoch 3/10
57/57 [==============================] - 3s 59ms/step - loss: 0.0047 - val_loss: 0.2295
Epoch 4/10
57/57 [==============================] - 3s 56ms/step - loss: 0.0033 - val_loss: 0.2643
Epoch 5/10
57/57 [==============================] - 3s 58ms/step - loss: 0.0013 - val_loss: 0.2640
Epoch 6/10
57/57 [==============================] - 3s 56ms/step - loss: 0.0022 - val_loss: 0.2717
Epoch 7/10
57/57 [==============================] - 3s 56ms/step - loss: 0.0038 - val_loss: 0.2412
Epoch 8/10
57/57 [==============================] - 3s 57ms/step - loss: 0.0032 - val_loss: 0.2677
Epoch 9/10
57/57 [==============================] - 3s 54ms/step - loss: 0.0018 - val_loss: 0.2625
Epoch 10/10
57/57 [==============================] - 3s 58ms/step - loss: 7.0211e-04 - val_loss: 0.2539


In [15]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(7734, 64)


In [0]:
import io


out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.tokens):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')